In [1]:
##implementando el metodo de la mexima verosimilirud para la regresion logistica

In [2]:
#definir la funcion de entorno L(b)

In [4]:
from IPython.display import display, Math, Latex
display(Math(r'L(\beta)=\sum_{i=1}^n P_i^{y_i}(1-Pi)^{y_i}'))

<IPython.core.display.Math object>

In [58]:
def likelihood(y,pi):
    import numpy
    total_sum=1
    sum_in=list(range(1,len(y)+1))
    for i in range(len(y)):
        sum_in[i]=numpy.where(y[i]==1,pi[i],1-pi[i])
        total_sum=total_sum*sum_in[i]
    return total_sum

In [7]:
#calcular las probabilidades para cada observacion

In [13]:
display(Math(r'P_i=P(x_i)=\frac{1}{1+e^{-\sum_{j=0}^k\beta\cdot x_{ij}}}'))

<IPython.core.display.Math object>

In [59]:
def logitprobs(x,beta):
    import numpy
    n_rows=numpy.shape(x)[0]
    n_cols=numpy.shape(x)[1]
    pi=list(range(1,n_rows+1))
    expon=list(range(1,n_rows+1))
    for i in range(n_rows):
        expon[i]=0
        for j in range(n_cols):
            ex=x[i][j]*beta[j]
            expon[i]=ex+expon[i]
        with numpy.errstate(divide="ignore",invalid="ignore"):
            pi[i]=1/(1+numpy.exp(-expon[i]))
    return pi

In [18]:
display(Math(r'W=diag(P_i \cdot (1-P_i))_{i=1}^n'))

<IPython.core.display.Math object>

In [61]:
def findW(pi):
    import numpy
    n=len(pi)
    w=numpy.zeros(n*n).reshape(n,n)
    for i in range(n):
        w[i,i]=pi[i]*(1-pi[i])
        w[i,i].astype(float)
    return w

In [20]:
#obtener la solucion de la funcion logistica

In [78]:
display(Math(r'\beta_{n+1}=\beta_n -\frac{f(\beta_n)}{f`(\beta_n)}'))
display(Math(r'f(x)=x(y-p)'))
display(Math(r'f`(x)=xwx^T'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [79]:
def logistics(x,y,limit):
    import numpy
    from numpy import linalg
    nrow=numpy.shape(x)[0]
    bias=numpy.ones(nrow).reshape(nrow,1)
    x_new=numpy.append(x,bias,axis=1)
    ncol=numpy.shape(x_new)[1]
    beta=numpy.zeros(ncol).reshape(ncol,1)
    root_dif=numpy.array(range(1,ncol+1)).reshape(ncol,1)
    iter_i=10000
    while(iter_i>limit):
        pi=logitprobs(x_new,beta)
        w=findW(pi)
        num=(numpy.transpose(numpy.matrix(x_new))*numpy.matrix(y-numpy.transpose(pi)).transpose())
        den=(numpy.matrix(numpy.transpose(x_new))*numpy.matrix(w)*numpy.matrix(x_new))
        root_dif=numpy.array(linalg.inv(den)*num)
        beta=beta+root_dif
        iter_i=numpy.sum(root_dif*root_dif)
        ll=likelihood(y,pi)
    return beta

In [37]:
#comprobacion experimental

In [8]:
import numpy
x=numpy.array(range(10)).reshape(10,1)

In [9]:
y=[0,0,0,0,1,0,1,0,1,1]

In [10]:
bias=numpy.ones(10).reshape(10,1)
x_new=numpy.append(x,bias,axis=1)

In [80]:
a=logistics(x,y,0.00001)

In [81]:
ll=likelihood(y,logitprobs(x,a))

In [82]:
ll

array([1.32622426e-06])

In [16]:
#y=(0.66220827)*X -3.69557172

In [11]:
#con el paquete statsmodels de python
import statsmodels.api as sm

In [12]:
logit_model=sm.Logit(y,x_new)

In [13]:
result=logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [15]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   10
Model:                          Logit   Df Residuals:                        8
Method:                           MLE   Df Model:                            1
Date:                Wed, 27 Jan 2021   Pseudo R-squ.:                  0.3596
Time:                        13:55:30   Log-Likelihood:                -4.3101
converged:                       True   LL-Null:                       -6.7301
Covariance Type:            nonrobust   LLR p-value:                   0.02781
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.6622      0.400      1.655      0.098      -0.122       1.446
const         -3.6956      2.289     -1.615      0.106      -8.182       0.791
==============================================================================
"""